In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch, torchvision
from torchvision.transforms import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import cv2

%matplotlib inline

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Test_agri.csv to Test_agri.csv
Saving train_agri.csv to train_agri.csv


In [3]:
import io
df_test = pd.read_csv(io.BytesIO(uploaded['Test_agri.csv']))
df_train = pd.read_csv(io.BytesIO(uploaded['train_agri.csv']))

In [4]:
df_train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0.0,0,1,0
1,F00000003,209,1,0,1,0,0.0,0,2,1
2,F00000004,257,1,0,1,0,0.0,0,2,1
3,F00000005,257,1,1,1,0,0.0,0,2,1
4,F00000006,342,1,0,1,0,0.0,0,2,1


In [5]:
df_train.isnull().sum()

ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          9000
Number_Weeks_Quit             0
Season                        0
Crop_Damage                   0
dtype: int64

In [6]:
df_train['Number_Weeks_Used'].value_counts()

20.0    3878
30.0    3765
25.0    3120
40.0    2707
15.0    2486
        ... 
63.0      50
64.0      44
65.0      31
66.0      17
67.0       8
Name: Number_Weeks_Used, Length: 64, dtype: int64

In [7]:
df_train['Number_Weeks_Used'] = df_train['Number_Weeks_Used'].fillna(df_train['Number_Weeks_Used'].median())

In [8]:
df_train['Number_Weeks_Used'].isnull().sum()

0

In [9]:
encoder = LabelEncoder()

In [10]:
df_train['ID'] = encoder.fit_transform(df_train['ID'])

In [11]:
df_test.isnull().sum()

ID                            0
Estimated_Insects_Count       0
Crop_Type                     0
Soil_Type                     0
Pesticide_Use_Category        0
Number_Doses_Week             0
Number_Weeks_Used          5893
Number_Weeks_Quit             0
Season                        0
dtype: int64

In [12]:
df_test['Number_Weeks_Used'] = df_test['Number_Weeks_Used'].fillna(df_test['Number_Weeks_Used'].median())

In [13]:
df_test.isnull().sum()

ID                         0
Estimated_Insects_Count    0
Crop_Type                  0
Soil_Type                  0
Pesticide_Use_Category     0
Number_Doses_Week          0
Number_Weeks_Used          0
Number_Weeks_Quit          0
Season                     0
dtype: int64

In [14]:
df_test['ID'] = encoder.fit_transform(df_test['ID'])

In [15]:
df_train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,0,188,1,0,1,0,0.0,0,1,0
1,1,209,1,0,1,0,0.0,0,2,1
2,2,257,1,0,1,0,0.0,0,2,1
3,3,257,1,1,1,0,0.0,0,2,1
4,4,342,1,0,1,0,0.0,0,2,1


In [16]:
df_test.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season
0,0,188,1,1,1,0,28.0,0,2
1,1,410,1,1,1,0,0.0,0,2
2,2,626,1,0,1,0,0.0,0,2
3,3,731,1,0,1,0,0.0,0,2
4,4,789,0,0,1,0,0.0,0,1


In [18]:
y = np.array(df_train['Crop_Damage'])

In [19]:
y

array([0, 1, 1, ..., 0, 0, 0])

In [20]:
X = np.array(df_train.iloc[:, df_train.columns != 'Crop_Damage'])

In [21]:
X

array([[0.0000e+00, 1.8800e+02, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.0000e+00, 2.0900e+02, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.0000e+00],
       [2.0000e+00, 2.5700e+02, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        2.0000e+00],
       ...,
       [8.8855e+04, 3.5160e+03, 1.0000e+00, ..., 4.0000e+01, 8.0000e+00,
        2.0000e+00],
       [8.8856e+04, 3.7020e+03, 1.0000e+00, ..., 2.5000e+01, 1.8000e+01,
        3.0000e+00],
       [8.8857e+04, 3.8950e+03, 1.0000e+00, ..., 3.7000e+01, 7.0000e+00,
        3.0000e+00]])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35)

In [23]:
X_train.shape

(57757, 9)

In [24]:
X_train

array([[2.0300e+02, 1.2120e+03, 0.0000e+00, ..., 4.7000e+01, 0.0000e+00,
        1.0000e+00],
       [6.0275e+04, 1.2960e+03, 0.0000e+00, ..., 2.8000e+01, 5.0000e+00,
        2.0000e+00],
       [7.3528e+04, 1.7850e+03, 1.0000e+00, ..., 2.8000e+01, 1.0000e+01,
        1.0000e+00],
       ...,
       [5.9915e+04, 5.3100e+02, 0.0000e+00, ..., 2.0000e+01, 1.0000e+01,
        2.0000e+00],
       [7.0258e+04, 1.1320e+03, 0.0000e+00, ..., 2.0000e+01, 1.8000e+01,
        2.0000e+00],
       [1.2308e+04, 3.3360e+03, 0.0000e+00, ..., 2.0000e+01, 3.1000e+01,
        1.0000e+00]])

In [25]:
y_train.shape

(57757,)

In [26]:
from torch.utils.data import DataLoader

In [27]:
batch_size = 100
train_dataloader = DataLoader(X_train, batch_size, shuffle = True)

In [28]:
for data in train_dataloader:
  print(data)
  break

tensor([[2.6668e+04, 2.9990e+03, 0.0000e+00, 0.0000e+00, 2.0000e+00, 2.0000e+01,
         2.8000e+01, 2.4000e+01, 1.0000e+00],
        [4.0283e+04, 2.5410e+03, 0.0000e+00, 1.0000e+00, 2.0000e+00, 1.0000e+01,
         1.8000e+01, 2.8000e+01, 2.0000e+00],
        [1.5057e+04, 9.8400e+02, 1.0000e+00, 0.0000e+00, 2.0000e+00, 3.0000e+01,
         4.1000e+01, 7.0000e+00, 1.0000e+00],
        [5.1460e+03, 2.0900e+02, 0.0000e+00, 1.0000e+00, 3.0000e+00, 4.0000e+01,
         1.0000e+01, 0.0000e+00, 2.0000e+00],
        [5.8129e+04, 4.1100e+02, 1.0000e+00, 0.0000e+00, 2.0000e+00, 5.0000e+00,
         5.0000e+00, 1.0000e+01, 2.0000e+00],
        [3.8002e+04, 1.2970e+03, 1.0000e+00, 0.0000e+00, 2.0000e+00, 2.0000e+01,
         4.2000e+01, 2.0000e+00, 1.0000e+00],
        [7.6768e+04, 1.7850e+03, 1.0000e+00, 0.0000e+00, 3.0000e+00, 4.0000e+01,
         2.8000e+01, 0.0000e+00, 1.0000e+00],
        [5.9132e+04, 2.0160e+03, 1.0000e+00, 0.0000e+00, 3.0000e+00, 1.0000e+01,
         1.5000e+01, 0.0000e+0

In [29]:
from torch.autograd import Variable

In [30]:
X_train = Variable(torch.from_numpy(X_train))
X_test = Variable(torch.from_numpy(X_test))
y_train = Variable(torch.from_numpy(y_train))
y_test = Variable(torch.from_numpy(y_test))

In [31]:
X_train

tensor([[2.0300e+02, 1.2120e+03, 0.0000e+00,  ..., 4.7000e+01, 0.0000e+00,
         1.0000e+00],
        [6.0275e+04, 1.2960e+03, 0.0000e+00,  ..., 2.8000e+01, 5.0000e+00,
         2.0000e+00],
        [7.3528e+04, 1.7850e+03, 1.0000e+00,  ..., 2.8000e+01, 1.0000e+01,
         1.0000e+00],
        ...,
        [5.9915e+04, 5.3100e+02, 0.0000e+00,  ..., 2.0000e+01, 1.0000e+01,
         2.0000e+00],
        [7.0258e+04, 1.1320e+03, 0.0000e+00,  ..., 2.0000e+01, 1.8000e+01,
         2.0000e+00],
        [1.2308e+04, 3.3360e+03, 0.0000e+00,  ..., 2.0000e+01, 3.1000e+01,
         1.0000e+00]], dtype=torch.float64)

In [32]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [33]:
y_train

tensor([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]])

In [72]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(9,32)
        self.l2 = torch.nn.Linear(32,64)
        self.l3 = torch.nn.Linear(64,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,1)
        self.drop = torch.nn.Dropout(0.25)
        
    def forward(self, x):
        x = F.relu(self.l1(x)) 
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        x = F.dropout(x)
        
        return torch.softmax(self.l5(x), dim = 0)
    
model = Model()
print(model)

Model(
  (l1): Linear(in_features=9, out_features=32, bias=True)
  (l2): Linear(in_features=32, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=128, bias=True)
  (l4): Linear(in_features=128, out_features=64, bias=True)
  (l5): Linear(in_features=64, out_features=1, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
)


In [74]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

criterion = nn.BCELoss()


for epoch in range(3):
  y_pred = model(X_train.float())
  loss = criterion(y_pred, y_train.float())

  # zero gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # updated for weights 
  optimizer.step()

print(loss)

tensor(19.1561, grad_fn=<BinaryCrossEntropyBackward>)


In [75]:
correct = 0
total = 0

with torch.no_grad():
  output = model(X_train.float())
  for idx, i in enumerate(output):
    if torch.argmax(i) == y[idx]:

      correct += 1
    total += 1

print("Accuracy : ", correct/total * 100)

Accuracy :  83.32669633118064


In [91]:
X_df_test = np.array(df_test)

In [92]:
X_df_test = Variable(torch.from_numpy(X_df_test))

In [93]:
y_pred = model(X_df_test.float())

In [94]:
y_pred.shape

torch.Size([59310, 1])